# An initial play with OpenAI and TAWA data

This TAWA output is from TAR 295, which modelled several single tier FTC changes, one of which was chosen for Budget 22.

In [49]:
from aipy.ai import *
import pandas as pd
import yaml
import jinja2
import tiktoken



## Available models

In [50]:
models = openai.Model.list().data

### GPT-4

In [51]:
[m['id'] for m in models if m['id'].startswith('gpt-4')]

['gpt-4',
 'gpt-4-vision-preview',
 'gpt-4-0314',
 'gpt-4-0613',
 'gpt-4-1106-preview']

### GPT-3.5

In [52]:
[m['id'] for m in models if m['id'].startswith('gpt-3.5')]

['gpt-3.5-turbo',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-1106',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-instruct-0914']

## Load data

tawa_details contains the tax year and the reform scenario descriptions

In [53]:
with open('tawa_details.yaml') as f:
    tawa_details = yaml.load(f, Loader=yaml.FullLoader)


In [54]:
tawa = pd.read_csv("input/tawa.csv")

tawa.drop(columns=['Name', 'Rounding_Rule', 'Population'], inplace=True)

In [55]:
fiscals = tawa[
    (tawa.Topic == 'Fiscals') & (tawa.Variable == 'Disposable_Income')][
        ['Scenario', 'Value', 'Margin_Of_Error']]

In [56]:
wnl_cols = [
    'Value', 'Margin_Of_Error', 'Scenario', 'Variable', 'Winner_Loser', 
    'Eq_DI_Quantile', 'WnL_Group']
wnl_vars = ['Mean_Weekly_Change', 'Population_In_Category']

wnl = tawa[(tawa.Topic == "WnL") & tawa.Variable.isin(wnl_vars)][wnl_cols]

In [57]:
poverty = tawa[
    (tawa.Topic == "Poverty") & (tawa.Variable == "Change_In_Population_In_Poverty")
    & (tawa.Population_Type == "Children")][
        ['Scenario', 'Poverty_Type', 'Value', 'Margin_Of_Error']]

## Prime model

In [58]:
with open('tawa_priming.jinja2') as f:
    priming_template = jinja2.Template(f.read())

In [59]:
chat = Chat()
chat.add_context(
    priming_template.render(
        tawa_details=tawa_details, fiscals=fiscals, wnl=wnl, poverty=poverty))

Display number of tokens in the context before asking any questions

In [60]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens



In [61]:
content = '\n'.join([m['content'] for m in chat.messages])

In [66]:
print(f"Number of tokens: {num_tokens_from_string(content, 'gpt-4')}")

Number of tokens: 30605


This number of tokens is so large that it only leaves the experimental 128k GPT-4
model as an option.  Note: the length of the context is dominated by the WnL tables
and could be reduced by dropping the separate Households with children etc.

## Ask some questions

In [ ]:
chat.ask(
    "From a value for money perspective, which reform best supports low income households?",
    model = "gpt-4-1106-preview")

In [69]:
print(chat.messages[-1]['content'])

From a value for money perspective, we would ideally look for the reform that provides the greatest positive impact on low-income households while incurring the least fiscal cost. To determine this, an examination of the cost of each reform compared to its impact on poverty reduction and household income changes, particularly for low-income groups, is necessary.

The reform that stands out is FTC5a42.7k27. It has the following characteristics:

- A lower fiscal cost of 67,875,000 compared to the other reforms, except for FTC5a42.7k26 which has a slightly lower fiscal cost but the same outcomes in terms of poverty reduction.
- It reduces the number of children living in both BHC and Fixed_AHC poverty by 5000 and 8000, respectively, which is a solid impact on reducing child poverty.
- The reform increases the FTC rate by 5 dollars per week and increases the abatement rate to 27%.

While the FTC7.5a42.7k27 reform impacts poverty more (reducing by 7000 under BHC and 11000 under Fixed_AHC),

funnily enough - that is the one they chose in Budget 22

In [ ]:
chat.ask("which reform is the most tightly targeted to low incomes?", model = "gpt-4-1106-preview")

In [73]:
print(chat.messages[-1]['content'])

To determine which reform is most tightly targeted to low-income households, we need to identify the reform that provides benefits to low-income groups while minimizing benefits to higher-income groups.

Analyzing the microsimulation output provided:

- FTC5a42.7k25: Increased the FTC rate by 5 dollars with no change to the abatement threshold or rate. It's a broad reform affecting all eligible families.
- FTC5a42.7k26: Same increase in FTC rate by 5 dollars, but the abatement rate is increased to 26%. This means that the benefit is reduced more quickly as income increases, which makes it more targeted to lower-income households compared to FTC5a42.7k25.
- FTC5a42.7k27: Further increases the abatement rate to 27%, which suggests that the benefit tapers off faster for higher incomes than in the FTC5a42.7k26 scenario, making it likely more targeted than the previous two options.
- FTC7.5a42.7k27: Increases the FTC rate more significantly, by 7.5 dollars per week, with a higher abatement 

it's chosen the same reform again, but it still hasn't explicitly made use of the
winners and losers data

In [ ]:
chat.ask("regarding your previous answer, can you refer to the number of winning and losing households in different HEDI deciles?", model = "gpt-4-1106-preview")

In [75]:
print(chat.messages[-1]['content'])

Certainly. To determine which reform is most tightly targeted to low incomes, let's look at the number of winning and losing households by equivalised disposable income (EQ_DI) deciles, focusing on the reforms with increased abatement rates:

**FTC5a42.7k26 with a 26% abatement rate:**
- Winning households are more numerous in the lower deciles, particularly in deciles 1 through 4, with populations of 52,000; 33,000; 75,000; and 69,000, respectively.
- Losers start appearing noticeably in decile 3 (3,000), with the number increasing in the higher deciles, especially in decile 4 (12,000).

**FTC5a42.7k27 with a 27% abatement rate:**
- Winning households span the lower deciles; we have significant numbers in deciles 1 through 3, with 51,000; 32,000; and 62,000, respectively.
- Losers appear more significantly in decile 4 (15,000) compared to FTC5a42.7k26, and at lower numbers in decile 3 (15,000).

**FTC7.5a42.7k27 with a 27% abatement rate but a higher increase in the FTC rate:**
- Winn

and a final check of the content tokens

In [76]:
content = '\n'.join([m['content'] for m in chat.messages])
print(f"Number of tokens: {num_tokens_from_string(content, 'gpt-4')}")

Number of tokens: 32113


In [77]:
chat.messages


[{'role': 'system', 'content': 'You are wise and helpful.'},
 {'role': 'user',
  'content': 'You will be asked questions that relate to microsimulation output for several hypothetical\nreforms to New Zealand\'s tax and transfer system in the 2022\ntax year. Your role will be to act as a chatbot and reply concisely.\n\nThe reform scenarios are described in the following dictionary:\n\n{\'FTC5a42.7k25\': \'Increase the Family Tax Credit (FTC) rate for the eldest child by 5 dollars per week, maintain the current abatement threshold at 42,700 dollars, and keep the abatement rate at 25%\', \'FTC5a42.7k26\': \'Increase FTC rate by 5 dollars per week and increase abatement rate to 26%\', \'FTC5a42.7k27\': \'Increase FTC rate by 5 dollars per week and increase abatement rate to 27%\', \'FTC7.5a42.7k27\': \'Increase FTC rate by 7.5 dollars per week and increase abatement rate to 27%\', \'FTC7.5a42.7k28\': \'Increase FTC rate by 7.5 dollars per week and increase abatement rate to 28%\'}\n\n\nThe